In [1]:
import json
from difflib import SequenceMatcher
import pandas as pd
import re

In [22]:
pre_df = pd.read_csv("./dataset_eval/Meta-Llama-3.2-3B-Instruct-lora-adapter-v4_output (8).csv")
gt_df = pd.read_csv("./dataset_eval/test_data.csv")

In [23]:
pre_df.head()

,input,model_output
0,* Domain: **Bán lẻ**\n* Chức năng: **Thanh...,<think>\n## 1. CHAIN ​​OF THOUGHT\n\n* **Ana...
1,Domain: Retail\nFunction: Add to Wishlist\nDes...,<think>\n## 1. CHAIN ​​OF THOUGHT\n\n* **Ana...
2,Domain: **Bán lẻ**\nChức năng: **Thêm vào giỏ ...,## 1. CHAIN ​​OF THOUGHT\n\n* **Business Goal ...
3,* Domain: **Bán lẻ**\n* Chức năng: **Thêm ...,<think>\n## 1. CHAIN ​​OF THOUGHT\n\n* **Bus...
4,Domain: Retail\nFunction: View Cart\nDescripti...,<think>\n## 1. CHAIN ​​OF THOUGHT\n\n* **Bus...


In [24]:
def extract_clean_json_from_text(raw_text):
    """
    Clean a single text block & extract JSON test case.
    Improved version with better Chinese comma handling and nested JSON extraction.
    """

    if pd.isna(raw_text):
        return None

    text = str(raw_text)

    # 1) Remove code fences
    text = re.sub(r"```json|```", "", text, flags=re.IGNORECASE)

    # 2) Remove <think> blocks
    # Remove <think>...</think> (có closing tag)
    text = re.sub(r"<think>.*?</think>", "", text, flags=re.DOTALL | re.IGNORECASE)
    # Remove <think>... (không có closing tag - xóa từ <think> đến hết)
    text = re.sub(r"<think>.*", "", text, flags=re.DOTALL | re.IGNORECASE)

    # 3) Find JSON blocks - sử dụng stack để tìm JSON block đúng nhất
    def find_json_block(text):
        """Tìm JSON block bằng cách đếm braces"""
        first_brace = text.find('{')
        if first_brace == -1:
            return None
        
        # Sử dụng stack để tìm } cuối cùng khớp với { đầu tiên
        stack = []
        in_string = False
        escape_next = False
        
        for i in range(first_brace, len(text)):
            char = text[i]
            
            if escape_next:
                escape_next = False
                continue
            
            if char == '\\':
                escape_next = True
                continue
            
            if char == '"' and not escape_next:
                in_string = not in_string
                continue
            
            if in_string:
                continue
            
            if char == '{':
                stack.append(i)
            elif char == '}':
                if stack:
                    stack.pop()
                    if len(stack) == 0:  # Đã đóng tất cả braces
                        return text[first_brace:i+1]
        
        # Nếu không tìm thấy closing brace đúng, thử tìm } cuối cùng
        last_brace = text.rfind('}', first_brace)
        if last_brace != -1:
            return text[first_brace:last_brace + 1]
        
        return None
    
    json_text = find_json_block(text)
    if json_text is None:
        return None
    
    # 4) Fix các vấn đề phổ biến trong JSON
    
    # 4.1) Thay thế dấu phẩy tiếng Trung (，) bằng dấu phẩy tiếng Anh (,)
    # Cần cẩn thận không thay trong string values
    # Cách làm: thay thế tất cả ， nhưng chỉ khi không nằm trong string
    def replace_chinese_commas(text):
        """Thay thế dấu phẩy Trung Quốc nhưng không thay trong string values"""
        result = []
        in_string = False
        escape_next = False
        
        i = 0
        while i < len(text):
            char = text[i]
            
            if escape_next:
                result.append(char)
                escape_next = False
                i += 1
                continue
            
            if char == '\\':
                result.append(char)
                escape_next = True
                i += 1
                continue
            
            if char == '"':
                in_string = not in_string
                result.append(char)
                i += 1
                continue
            
            if char == '，' and not in_string:
                result.append(',')
            else:
                result.append(char)
            
            i += 1
        
        return ''.join(result)
    
    json_text = replace_chinese_commas(json_text)
    
    # 4.2) Xử lý escape characters (nếu cần)
    # Thường không cần vì JSON parser tự xử lý, nhưng nếu có vấn đề thì thử
    try:
        # Thử parse trực tiếp trước
        data = json.loads(json_text)
    except json.JSONDecodeError as e:
        # Nếu lỗi, thử fix thêm
        # Loại bỏ trailing commas (dấu phẩy thừa ở cuối)
        json_text = re.sub(r',\s*}', '}', json_text)
        json_text = re.sub(r',\s*]', ']', json_text)
        
        # Thử lại
        try:
            data = json.loads(json_text)
        except json.JSONDecodeError as e2:
            # Nếu vẫn lỗi, thử tìm JSON block khác
            # Tìm tất cả các JSON blocks có thể
            candidates = []
            start = 0
            while True:
                start_brace = text.find('{', start)
                if start_brace == -1:
                    break
                block = find_json_block(text[start_brace:])
                if block:
                    candidates.append(block)
                start = start_brace + 1
            
            if candidates:
                # Thử từng candidate, chọn cái parse được
                for candidate in sorted(candidates, key=len, reverse=True):
                    candidate = replace_chinese_commas(candidate)
                    candidate = re.sub(r',\s*}', '}', candidate)
                    candidate = re.sub(r',\s*]', ']', candidate)
                    try:
                        data = json.loads(candidate)
                        json_text = candidate
                        break
                    except:
                        continue
                else:
                    return None
            else:
                return None

    # 5) Only keep what we need
    cleaned_data = {
        "request_body": data.get("request_body", {}),
        "testcases": data.get("testcases", {})
    }

    # 6) Validate testcases structure, specifically response_mapping
    # Ensure that `response_mapping` within any test case is a dictionary.
    testcases = cleaned_data.get("testcases")
    if testcases:
        for tc_type in ["basic_validation", "business_logic"]:
            if tc_type in testcases and isinstance(testcases[tc_type], list):
                for tc_entry in testcases[tc_type]:
                    expected_output = tc_entry.get("expected_output")
                    if expected_output and "response_mapping" in expected_output:
                        resp_mapping = expected_output["response_mapping"]
                        # Chỉ validate nếu response_mapping không phải None và không phải dict
                        if resp_mapping is not None and not isinstance(resp_mapping, dict):
                            # If response_mapping is not a dict (và không phải None), consider this entire JSON invalid
                            return None

    return cleaned_data

In [25]:
# Extract JSON từ model_output
print("Đang trích xuất JSON từ model_output...")
pre_df["clean_output"] = pre_df["model_output"].apply(extract_clean_json_from_text)
gt_df["clean_output"] = gt_df["json_output"].apply(extract_clean_json_from_text)

# Kiểm tra kết quả
print(f"\n📊 Kết quả trích xuất:")
print(f"   - Tổng số records: {len(pre_df)}")
print(f"   - Records có JSON hợp lệ: {pre_df['clean_output'].notna().sum()}")
print(f"   - Records không trích xuất được: {pre_df['clean_output'].isna().sum()}")
print(f"   - Tỷ lệ thành công: {pre_df['clean_output'].notna().sum() / len(pre_df) * 100:.2f}%")

# Hiển thị một vài records không trích xuất được (nếu có)
failed_indices = pre_df[pre_df['clean_output'].isna()].index[:3]
if len(failed_indices) > 0:
    print(f"\n⚠️  Một vài records không trích xuất được (index): {list(failed_indices)}")
    print("   Bạn có thể kiểm tra model_output của các records này để debug.")

Đang trích xuất JSON từ model_output...

📊 Kết quả trích xuất:
   - Tổng số records: 129
   - Records có JSON hợp lệ: 11
   - Records không trích xuất được: 118
   - Tỷ lệ thành công: 8.53%

⚠️  Một vài records không trích xuất được (index): [0, 1, 2]
   Bạn có thể kiểm tra model_output của các records này để debug.


In [31]:
# Debug: Kiểm tra một vài records thất bại
import json

def debug_extraction(idx):
    """Debug extraction cho một record cụ thể"""
    print(f"\n{'='*60}")
    print(f"DEBUG RECORD #{idx}")
    print(f"{'='*60}")
    
    model_output = pre_df.iloc[idx]['model_output']
    print(f"\n📝 Model output (first 500 chars):")
    print(model_output[:500])
    print("...")
    
    # Thử extract
    result = extract_clean_json_from_text(model_output)
    
    if result is None:
        print("\n❌ Không trích xuất được JSON")
        
        # Tìm { đầu tiên
        first_brace = model_output.find('{')
        if first_brace != -1:
            print(f"\n✅ Tìm thấy {{ đầu tiên tại vị trí: {first_brace}")
            print(f"   Text xung quanh: ...{model_output[max(0, first_brace-50):first_brace+200]}...")
            
            # Thử tìm JSON block thủ công
            text = str(model_output)
            text = re.sub(r"```json|```", "", text, flags=re.IGNORECASE)
            text = re.sub(r"<think>.*?</think>", "", text, flags=re.DOTALL | re.IGNORECASE)
            text = re.sub(r"<think>.*", "", text, flags=re.DOTALL | re.IGNORECASE)
            
            first_brace = text.find('{')
            if first_brace != -1:
                last_brace = text.rfind('}', first_brace)
                if last_brace != -1:
                    json_candidate = text[first_brace:last_brace + 1]
                    print(f"\n📋 JSON candidate (first 300 chars):")
                    print(json_candidate[:300])
                    print("...")
                    
                    # Thử parse
                    try:
                        # Thay dấu phẩy Trung Quốc
                        json_candidate_fixed = json_candidate.replace('，', ',')
                        data = json.loads(json_candidate_fixed)
                        print("\n✅ JSON candidate có thể parse được sau khi fix!")
                    except json.JSONDecodeError as e:
                        print(f"\n❌ JSON candidate không parse được: {e}")
                        print(f"   Lỗi tại vị trí: {e.pos}")
                        if e.pos:
                            start = max(0, e.pos - 50)
                            end = min(len(json_candidate_fixed), e.pos + 50)
                            print(f"   Context: ...{json_candidate_fixed[start:end]}...")
        else:
            print("\n❌ Không tìm thấy { trong model_output")
    else:
        print("\n✅ Trích xuất thành công!")
        print(f"   Keys: {list(result.keys())}")
        if 'testcases' in result:
            testcases = result['testcases']
            if isinstance(testcases, dict):
                for key in testcases:
                    if isinstance(testcases[key], list):
                        print(f"   {key}: {len(testcases[key])} test cases")
    
    return result

# Debug 3 records đầu tiên
for idx in [0, 1, 2]:
    debug_extraction(idx)



DEBUG RECORD #0

📝 Model output (first 500 chars):
<think>\n## 1. CHAIN ​​OF THOUGHT\n\n*   **Analyze Business Objectives:** "First， what does this API exist for? What is the main goal? (E.g.， 'Allow the user to create a new order'). Important assumptions， preconditions What is important? (For example: 'User must have admin rights'， 'Product must exist in stock')."\n    *   Mục tiêu chính của API này là xử lý thanh toán cho một đơn hàng cụ thể.\n    *   Các giả định quan trọng là đơn hàng (`orderId`) phải tồn tại và có trạng thái hợp lệ để thanh
...

❌ Không trích xuất được JSON

✅ Tìm thấy { đầu tiên tại vị trí: 3577
   Text xung quanh: ...rror case for an invalid order status.\n</think>\n{\n  "request_body": {\n    "orderId": "ORD-2024-001"，\n    "paymentMethodId": "PM-VISA-1234"，\n    "amount": 1500000，\n    "currency": "VND"\n  }，\n  "testcases": {\n    "basic_validation": [\n      ...

📋 JSON candidate (first 300 chars):
{\n  "request_body": {\n    "orderId": "ORD-2024-001"，\n   

In [27]:
pre_df["request_body"] = pre_df["clean_output"].apply(lambda x: x.get("request_body") if x else None)
pre_df["testcases"] = pre_df["clean_output"].apply(lambda x: x.get("testcases") if x else None)

gt_df["request_body"] = gt_df["clean_output"].apply(lambda x: x.get("request_body") if x else None)
gt_df["testcases"] = gt_df["clean_output"].apply(lambda x: x.get("testcases") if x else None)

In [28]:
pre_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   input         129 non-null    object
 1   model_output  129 non-null    object
 2   clean_output  11 non-null     object
 3   request_body  11 non-null     object
 4   testcases     11 non-null     object
dtypes: object(5)
memory usage: 5.2+ KB


In [30]:
pre_df['testcases'][0]

In [ ]:
pre_df['testcases'][1]

# 📋 CÁCH TÍNH TP, FP, FN DỰA TRÊN MATCHING + PARTIAL MATCHING

## 1. Các trường hợp tính TP, FP, FN

### 1.1. Trường hợp Model JSON lỗi (parse lỗi)
- **Tất cả test cases trong GT** → **FN**
- **TP = 0, FP = 0, FN = số lượng test cases trong GT**

### 1.2. Trường hợp Model có test case mà GT không có
- **FP += 1**
- Status: `'FP'`
- Reason: `'Model has extra test case'`

### 1.3. Trường hợp GT có test case mà Model không có
- **FN += 1**
- Status: `'FN'`
- Reason: `'Model missing test case'`

### 1.4. Trường hợp cả GT và Model đều có test case → So sánh

#### Bước 1: Tính similarity
- Sử dụng hàm `compare_cases(gt_case, model_case, similarity_threshold=0.85)`
- Trả về: `(is_match, similarity, field_details)`
- `similarity`: giá trị từ 0.0 đến 1.0

#### Bước 2: Phân loại dựa trên similarity

**a) Match hoàn toàn (is_match = True, similarity >= threshold)**
- **TP += 1**
- Status: `'TP'`
- Reason: `'Match (similarity: X.XX)'`

**b) Partial match (is_match = False, nhưng similarity >= 0.7)**
- **TP += 0.5**
- **FP += 0.5**
- **FN += 0.5**
- Status: `'PARTIAL'`
- Reason: `'Partial match (similarity: X.XX)'`

**c) Không match (is_match = False, similarity < 0.7)**
- **FP += 1**
- **FN += 1**
- Status: `'FP+FN'`
- Reason: `'Low similarity (similarity: X.XX)'`

---

## 2. Công thức tính Precision, Recall, F1

Sau khi tính xong TP, FP, FN cho tất cả test cases trong 1 record:

### 2.1. Precision (Độ chính xác)
```
Precision = TP / (TP + FP)
```
- **Ý nghĩa**: Trong số các test case model dự đoán, bao nhiêu là đúng?
- **Nếu TP + FP = 0** → Precision = 0.0

### 2.2. Recall (Độ bao phủ)
```
Recall = TP / (TP + FN)
```
- **Ý nghĩa**: Trong số các test case cần có, model tìm được bao nhiêu?
- **Nếu TP + FN = 0** → Recall = 0.0

### 2.3. F1 Score (Cân bằng Precision và Recall)
```
F1 = 2 × (Precision × Recall) / (Precision + Recall)
```
- **Ý nghĩa**: Harmonic mean của Precision và Recall
- **Nếu Precision + Recall = 0** → F1 = 0.0

---

## 3. Ví dụ cụ thể

### Ví dụ 1: Record có 17 test cases

**Kết quả so sánh:**
- 8 test cases: TP (match hoàn toàn)
- 8 test cases: FP+FN (similarity < 0.7)
- 1 test case: FN (model thiếu)

**Tính toán:**
- TP = 8.0
- FP = 8.0 (từ 8 test cases FP+FN)
- FN = 9.0 (8 từ FP+FN + 1 từ missing)

**Metrics:**
- Precision = 8.0 / (8.0 + 8.0) = 0.5000
- Recall = 8.0 / (8.0 + 9.0) = 0.4706
- F1 = 2 × (0.5000 × 0.4706) / (0.5000 + 0.4706) = 0.4848

### Ví dụ 2: Record có partial match

**Kết quả so sánh:**
- 5 test cases: TP (match hoàn toàn)
- 3 test cases: PARTIAL (similarity >= 0.7 nhưng < 0.85)
- 2 test cases: FP+FN (similarity < 0.7)
- 1 test case: FN (model thiếu)

**Tính toán:**
- TP = 5.0 + (3 × 0.5) = 6.5
- FP = (3 × 0.5) + 2.0 = 3.5
- FN = (3 × 0.5) + 2.0 + 1.0 = 4.5

**Metrics:**
- Precision = 6.5 / (6.5 + 3.5) = 0.6500
- Recall = 6.5 / (6.5 + 4.5) = 0.5909
- F1 = 2 × (0.6500 × 0.5909) / (0.6500 + 0.5909) = 0.6190

---

## 4. Ngưỡng similarity

- **similarity_threshold = 0.85**: Ngưỡng để coi là match hoàn toàn
- **partial_threshold = 0.7**: Ngưỡng để coi là partial match
- **similarity < 0.7**: Không match, tính cả FP và FN

---

## 5. Tổng hợp metrics

Sau khi tính metrics cho từng record, có thể tính **Overall Metrics**:

```python
total_tp = sum([r['tp'] for r in detailed_results])
total_fp = sum([r['fp'] for r in detailed_results])
total_fn = sum([r['fn'] for r in detailed_results])

overall_precision = total_tp / (total_tp + total_fp)
overall_recall = total_tp / (total_tp + total_fn)
overall_f1 = 2 * (overall_precision * overall_recall) / (overall_precision + overall_recall)
```


# BÁO CÁO ĐÁNH GIÁ HỆ THỐNG TEST CASE MODEL

## 1. Mục tiêu của hệ thống đánh giá

Hệ thống đánh giá test case do bạn xây dựng nhằm:

1. So sánh **Test Case do LLM sinh ra** với **Ground Truth (GT)**.
2. Xác định **đúng/sai**, đồng thời tính **precision, recall, F1**.
3. Cho phép **so sánh linh hoạt** (flexible matching) thay vì yêu cầu exact match tuyệt đối.
4. Lưu **chi tiết từng field**, giúp debug model hiệu quả.

---

## 2. Nguyên tắc đánh giá

### 2.1. Flatten Test Case
- Tất cả các test case được flatten thành dạng key-value:
    - `req_<field>`: input request  
    - `exp_<field>`: expected output  
    - `res_<field>`: response_mapping  
- Mục tiêu: đảm bảo so sánh **từng trường dữ liệu** mà không bị lỗi nested dict.

### 2.2. Xử lý NULL / ABSENT / N/A
- GT có giá trị `NULL`, `ABSENT` hoặc `N/A` → model phải match **exact string hoặc None/empty**.
- Nếu model trả giá trị khác → **FAIL**, không cho partial match.

| GT | Model | Kết quả |
|----|-------|---------|
| "NULL" | None | ✔ PASS |
| "NULL" | "NULL" | ✔ PASS |
| "ABSENT" | field missing | ✔ PASS |
| "ABSENT" | value khác | ❌ FAIL |
| "N/A" | "N/A" | ✔ PASS |

### 2.3. So sánh dữ liệu số
- Nếu là số → convert sang float.
- Exact match → 1.0 similarity.
- Sai số ≤ 1% → partial match, similarity = 1 - diff.
- Sai số > 1% → similarity giảm.

### 2.4. So sánh dữ liệu string
- Normalize: lowercase, trim, bỏ multiple spaces.
- Fuzzy matching (SequenceMatcher) → similarity (0-1).
- Threshold 0.85 → match, <0.85 → partial hoặc fail.

### 2.5. Exact match fallback
- Nếu không phải số, không phải string → so sánh exact match.
- Không match → similarity = 0.

---

## 3. Tính toán similarity tổng thể
- Tổng similarity = **trung bình trọng số của các field**.
- Có thể đặt `field_weights` để ưu tiên một số field quan trọng hơn.
- `overall_similarity >= threshold` → testcase coi là **match**.

---

## 4. Xử lý các trường hợp đặc biệt
- Model JSON lỗi (parse lỗi) → tất cả test case → **FN**.
- Test case GT có mà Model không có → **FN**.
- Test case Model có mà GT không có → **FP**.
- Partial match (similarity ≥ 0.7) → TP = 0.5, FP = 0.5, FN = 0.5.

---

## 5. Output chi tiết
Mỗi testcase lưu chi tiết:

```json
{
    "test_case_idx": 0,
    "status": "TP / FP / FN / PARTIAL",
    "similarity": 0.87,
    "reason": "Match hoặc lý do fail",
    "field_details": {
        "req_userId": {
            "match": True,
            "similarity": 1.0,
            "reason": "Strings similar",
            "gt_value": "usr_123",
            "model_value": "usr_123"
        },
        "res_status": {
            "match": False,
            "similarity": 0.0,
            "reason": "Model trả khác",
            "gt_value": "ACTIVE",
            "model_value": "PENDING"
        }
    }
}

---

## 6. Tính metrics
- TP, FP, FN: dựa trên matching + partial matching.
- Precision = TP / (TP + FP)
- Recall = TP / (TP + FN)
- F1 = 2 * (precision * recall) / (precision + recall)
- Tính trung bình similarity để đánh giá tổng thể.

In [ ]:
def normalize_string(s):
    """Chuẩn hóa string để so sánh: lowercase, bỏ whitespace thừa"""
    if s is None:
        return ""
    s = str(s).strip().lower()
    # Bỏ multiple spaces
    s = ' '.join(s.split())
    return s

def calculate_string_similarity(str1, str2):
    """Tính similarity giữa 2 strings (0-1)"""
    if str1 is None and str2 is None:
        return 1.0
    if str1 is None or str2 is None:
        return 0.0
    
    str1_norm = normalize_string(str1) 
    str2_norm = normalize_string(str2) 
    
    if str1_norm == str2_norm:
        return 1.0
    
    # Sử dụng SequenceMatcher cho fuzzy matching
    return SequenceMatcher(None, str1_norm, str2_norm).ratio()

def compare_values(gt_val, model_val, field_type="auto"):
    """
    So sánh 2 giá trị một cách linh hoạt.
    
    Args:
        gt_val: Ground truth value
        model_val: Model predicted value
        field_type: "auto", "string", "number", "exact"
    
    Returns:
        (is_match: bool, similarity: float, reason: str)
    """
    # Xử lý NULL/ABSENT/N/A
    if gt_val in ["NULL", "ABSENT", "N/A"]:
        if model_val in [None, "NULL", "ABSENT", "N/A", ""]:
            return True, 1.0, "Both are NULL/ABSENT"
        return False, 0.0, f"GT requires NULL but model has: {model_val}"
    
    if model_val is None:
        return False, 0.0, "Model value is None"
    
    # Auto-detect type
    if field_type == "auto":
        if isinstance(gt_val, (int, float)) or isinstance(model_val, (int, float)):
            field_type = "number"
        elif isinstance(gt_val, str) and isinstance(model_val, str):
            field_type = "string"
        else:
            field_type = "exact"
    
    # So sánh số
    if field_type == "number":
        try:
            gt_num = float(gt_val)
            model_num = float(model_val)
            if gt_num == model_num:
                return True, 1.0, "Numbers match exactly"
            # Cho phép sai số nhỏ (1%)
            diff = abs(gt_num - model_num) / max(abs(gt_num), 1)
            if diff < 0.01:
                return True, 1.0 - diff, "Numbers match approximately"
            return False, 1.0 - min(diff, 1.0), f"Numbers differ: {gt_num} vs {model_num}"
        except (ValueError, TypeError):
            # Không phải số, so sánh như string
            field_type = "string"
    
    # So sánh string
    if field_type == "string":
        similarity = calculate_string_similarity(gt_val, model_val)
        # Threshold: >= 0.85 được coi là match
        threshold = 0.85
        if similarity >= threshold:
            return True, similarity, f"Strings similar (similarity: {similarity:.2f})"
        return False, similarity, f"Strings differ (similarity: {similarity:.2f})"
    
    # Exact match (fallback)
    if gt_val == model_val:
        return True, 1.0, "Exact match"
    return False, 0.0, f"Values differ: {gt_val} vs {model_val}"

def flatten_case(tc):
    """Flatten test case để so sánh"""
    flat = {}
    
    flat["test_case"] = tc.get("test_case", "")
    
    # request_mapping
    for k, v in tc.get("request_mapping", {}).items():
        flat[f"req_{k}"] = v
    
    # expected_output
    for k, v in tc.get("expected_output", {}).items():
        if k == "response_mapping":
            # flatten response_mapping
            for kk, vv in v.items():
                flat[f"res_{kk}"] = vv
        else:
            flat[f"exp_{k}"] = v
    
    return flat

def compare_cases(gt_case, model_case, similarity_threshold=0.85, field_weights=None):
    """
    So sánh 2 test cases một cách linh hoạt.
    
    Args:
        gt_case: Ground truth test case
        model_case: Model predicted test case
        similarity_threshold: Ngưỡng similarity tổng thể để coi là match (0-1)
        field_weights: Dict với weights cho từng field (optional)
    
    Returns:
        (is_match: bool, overall_similarity: float, details: dict)
    """
    if field_weights is None:
        field_weights = {}
    
    gt_flat = flatten_case(gt_case)
    model_flat = flatten_case(model_case)
    
    total_similarity = 0.0
    total_weight = 0.0
    field_details = {}
    all_match = True
    
    # So sánh từng field
    for key in gt_flat:
        gt_val = gt_flat[key]
        model_val = model_flat.get(key, None)
        
        # Xác định field type
        field_type = "auto"
        if key == "exp_statuscode" or "statuscode" in key.lower():
            field_type = "number"
        elif "test_case" in key:
            field_type = "string"
        
        # So sánh
        is_match, similarity, reason = compare_values(gt_val, model_val, field_type)
        
        weight = field_weights.get(key, 1.0)
        total_similarity += similarity * weight
        total_weight += weight
        
        field_details[key] = {
            'match': is_match,
            'similarity': similarity,
            'reason': reason,
            'gt_value': gt_val,
            'model_value': model_val
        }
        
        if not is_match:
            all_match = False
    
    # Tính overall similarity
    if total_weight > 0:
        overall_similarity = total_similarity / total_weight
    else:
        overall_similarity = 0.0
    
    # Kiểm tra các field model có mà GT không có (không tính vào similarity nhưng ghi nhận)
    extra_fields = set(model_flat.keys()) - set(gt_flat.keys())
    if extra_fields:
        field_details['_extra_fields'] = list(extra_fields)
    
    # Quyết định match dựa trên threshold
    final_match = overall_similarity >= similarity_threshold
    
    return final_match, overall_similarity, field_details


In [ ]:
# ================================================================
# 6b. CẬP NHẬT HÀM calculate_testcase_metrics VỚI FUZZY MATCHING
# ================================================================

def calculate_testcase_metrics_flexible(gt_json, model_json, similarity_threshold=0.85):
    """
    Tính precision, recall, F1 cho từng test case với Fuzzy Matching linh hoạt.
    
    Logic:
    - Với mỗi test case GT: so sánh với TẤT CẢ test cases Model
    - Chọn Model case có similarity cao nhất
    - Nếu similarity >= threshold → match (TP)
    - Nếu similarity < threshold → GT đó là FN
    - Model cases không được ghép → FP
    
    Args:
        gt_json: Ground truth JSON
        model_json: Model prediction JSON
        similarity_threshold: Ngưỡng similarity để coi là match (default: 0.85)
    
    Returns:
        dict với keys: 'tp', 'fp', 'fn', 'precision', 'recall', 'f1', 'details'
    """
    result = {
        'tp': 0.0,
        'fp': 0.0,
        'fn': 0.0,
        'precision': 0.0,
        'recall': 0.0,
        'f1': 0.0,
        'details': [],
        'avg_similarity': 0.0
    }
    
    # Nếu model JSON lỗi → tất cả là FN
    if model_json is None:
        gt_cases = gt_json.get("basic_validation", []) if gt_json else []
        result['fn'] = len(gt_cases)
        for idx, gt_case in enumerate(gt_cases):
            result['details'].append({
                'test_case_idx': idx,
                'status': 'FN',
                'reason': 'Model JSON is None',
                'similarity': 0.0,
                'matched_model_idx': None
            })
        return result
    
    gt_cases = gt_json.get("basic_validation", []) if gt_json else []
    model_cases = model_json.get("basic_validation", []) if model_json else []
    
    # Khởi tạo: tất cả Model cases chưa được match
    model_matched = [False] * len(model_cases)
    total_similarity = 0.0
    matched_count = 0
    
    # Với mỗi GT case: tìm Model case có similarity cao nhất (chưa được match)
    for gt_idx, gt_case in enumerate(gt_cases):
        best_similarity = 0.0
        best_model_idx = None
        best_field_details = None
        best_is_match = False
        
        # So sánh với tất cả Model cases (chỉ xét các case chưa được match)
        for model_idx, model_case in enumerate(model_cases):
            # Bỏ qua Model case đã được match
            if model_matched[model_idx]:
                continue
                
            # Tính similarity
            is_match, similarity, field_details = compare_cases(
                gt_case, model_case, 
                similarity_threshold=similarity_threshold
            )
            
            # Chọn Model case có similarity cao nhất
            if similarity > best_similarity:
                best_similarity = similarity
                best_model_idx = model_idx
                best_field_details = field_details
                best_is_match = is_match
        
        # Tạo detail cho GT case này
        detail = {
            'test_case_idx': gt_idx,
            'status': None,
            'reason': None,
            'similarity': best_similarity,
            'field_details': best_field_details,
            'matched_model_idx': best_model_idx
        }
        
        # Quyết định match dựa trên threshold
        if best_model_idx is not None and best_similarity >= similarity_threshold:
            # Match thành công → TP
            result['tp'] += 1
            detail['status'] = 'TP'
            detail['reason'] = f'Match (similarity: {best_similarity:.4f})'
            model_matched[best_model_idx] = True
            total_similarity += best_similarity
            matched_count += 1
        else:
            # Không match đủ tốt → FN
            result['fn'] += 1
            if best_model_idx is None:
                detail['status'] = 'FN'
                detail['reason'] = 'No available Model case to match (all already matched)'
            else:
                detail['status'] = 'FN'
                detail['reason'] = f'No match found (best similarity: {best_similarity:.4f} < {similarity_threshold})'
            # Vẫn ghi nhận similarity để phân tích
            if best_similarity > 0:
                total_similarity += best_similarity
                matched_count += 1
        
        result['details'].append(detail)
    
    # Model cases không được ghép → FP
    for model_idx, is_matched in enumerate(model_matched):
        if not is_matched:
            result['fp'] += 1
            result['details'].append({
                'test_case_idx': None,  # Không có GT tương ứng
                'status': 'FP',
                'reason': f'Model test case #{model_idx} not matched with any GT',
                'similarity': 0.0,
                'field_details': None,
                'matched_model_idx': model_idx
            })
    
    # Tính trung bình similarity
    if matched_count > 0:
        result['avg_similarity'] = total_similarity / matched_count
    
    # Tính precision, recall, F1
    tp = result['tp']
    fp = result['fp']
    fn = result['fn']
    
    # Precision = TP / (TP + FP)
    if tp + fp > 0:
        result['precision'] = tp / (tp + fp)
    else:
        result['precision'] = 0.0
    
    # Recall = TP / (TP + FN)
    if tp + fn > 0:
        result['recall'] = tp / (tp + fn)
    else:
        result['recall'] = 0.0
    
    # F1 = 2 * (precision * recall) / (precision + recall)
    if result['precision'] + result['recall'] > 0:
        result['f1'] = 2 * (result['precision'] * result['recall']) / (result['precision'] + result['recall'])
    else:
        result['f1'] = 0.0
    
    return result

# Sử dụng hàm mới thay cho hàm cũ
# calculate_testcase_metrics = calculate_testcase_metrics_flexible


In [ ]:
# ================================================================
# THAY THẾ HÀM CŨ BẰNG HÀM MỚI (LINH HOẠT)
# ================================================================

# Uncomment dòng dưới để sử dụng hàm so sánh linh hoạt
calculate_testcase_metrics = calculate_testcase_metrics_flexible

print("✅ Đã chuyển sang sử dụng Fuzzy Matching linh hoạt!")
print("   - Với mỗi GT test case: so sánh với TẤT CẢ Model test cases")
print("   - Chọn Model case có similarity cao nhất")
print("   - Nếu similarity >= threshold (0.85) → match (TP)")
print("   - Nếu similarity < threshold → GT đó là FN")
print("   - Model cases không được ghép → FP")
print("   - Fuzzy matching cho strings + field matching")
print("   - Tính similarity score chi tiết cho từng field")


In [ ]:
# ================================================================
# 7. ĐÁNH GIÁ TẤT CẢ RECORDS VÀ TÍNH METRICS
# ================================================================

# Tính metrics cho từng record
detailed_results = []
for idx in range(len(gt_df)):
    gt_json = gt_df.iloc[idx]["testcases"]
    model_json = pre_df.iloc[idx]["testcases"]
    
    metrics = calculate_testcase_metrics(gt_json, model_json)
    metrics['record_idx'] = idx
    metrics['record_id'] = gt_df.iloc[idx].get('id', idx)
    detailed_results.append(metrics)

# Thêm metrics vào dataframe
pre_df['metrics'] = detailed_results
pre_df['precision'] = [r['precision'] for r in detailed_results]
pre_df['recall'] = [r['recall'] for r in detailed_results]
pre_df['f1'] = [r['f1'] for r in detailed_results]
pre_df['tp'] = [r['tp'] for r in detailed_results]
pre_df['fp'] = [r['fp'] for r in detailed_results]
pre_df['fn'] = [r['fn'] for r in detailed_results]


In [ ]:
# ================================================================
# 8. HIỂN THỊ KẾT QUẢ TỔNG QUAN
# ================================================================

# Tính tổng TP, FP, FN
total_tp = sum([r['tp'] for r in detailed_results])
total_fp = sum([r['fp'] for r in detailed_results])
total_fn = sum([r['fn'] for r in detailed_results])

# Tính overall precision, recall, F1
overall_precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0.0
overall_recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0.0
overall_f1 = 2 * (overall_precision * overall_recall) / (overall_precision + overall_recall) if (overall_precision + overall_recall) > 0 else 0.0

# Tính trung bình precision, recall, F1
avg_precision = pre_df['precision'].mean()
avg_recall = pre_df['recall'].mean()
avg_f1 = pre_df['f1'].mean()

print("=" * 60)
print("KẾT QUẢ ĐÁNH GIÁ TỔNG QUAN")
print("=" * 60)
print(f"\n📊 Số lượng test cases:")
print(f"   - True Positives (TP):  {total_tp}")
print(f"   - False Positives (FP): {total_fp}")
print(f"   - False Negatives (FN): {total_fn}")
print(f"   - Tổng GT test cases:   {total_tp + total_fn}")
print(f"   - Tổng Pred test cases: {total_tp + total_fp}")

print(f"\n📈 Overall Metrics (tính từ tổng TP/FP/FN):")
print(f"   - Precision: {overall_precision:.4f}")
print(f"   - Recall:    {overall_recall:.4f}")
print(f"   - F1 Score:  {overall_f1:.4f}")

print(f"\n📊 Average Metrics (trung bình từng record):")
print(f"   - Precision: {avg_precision:.4f}")
print(f"   - Recall:    {avg_recall:.4f}")
print(f"   - F1 Score:  {avg_f1:.4f}")

print(f"\n📋 Số records:")
print(f"   - Tổng số records: {len(pre_df)}")
print(f"   - Records có JSON hợp lệ: {pre_df['testcases'].notna().sum()}")
print(f"   - Records có JSON lỗi: {pre_df['testcases'].isna().sum()}")


In [ ]:
# ================================================================
# 9. XEM CHI TIẾT TỪNG RECORD
# ================================================================

# Hiển thị một số records với metrics
display_cols = ['id', 'precision', 'recall', 'f1', 'tp', 'fp', 'fn']
print("\n" + "=" * 60)
print("CHI TIẾT MỘT SỐ RECORDS (top 10)")
print("=" * 60)
print(pre_df[display_cols].head(10).to_string())


In [ ]:
# ================================================================
# 10. XEM CHI TIẾT TỪNG TEST CASE TRONG 1 RECORD
# ================================================================

def show_record_details(record_idx):
    """
    Hiển thị chi tiết từng test case trong 1 record.
    """
    if record_idx >= len(detailed_results):
        print(f"Record index {record_idx} không tồn tại!")
        return
    
    result = detailed_results[record_idx]
    print(f"\n{'='*60}")
    print(f"CHI TIẾT RECORD #{record_idx} (ID: {result['record_id']})")
    print(f"{'='*60}")
    print(f"\n📊 Metrics:")
    print(f"   - Precision: {result['precision']:.4f}")
    print(f"   - Recall:    {result['recall']:.4f}")
    print(f"   - F1 Score:  {result['f1']:.4f}")
    print(f"   - TP: {result['tp']}, FP: {result['fp']}, FN: {result['fn']}")
    
    print(f"\n📋 Chi tiết từng test case:")
    for detail in result['details']:
        status_emoji = {
            'TP': '✅',
            'FP': '❌',
            'FN': '⚠️',
            'FP+FN': '❌⚠️',
            'PARTIAL': '🟡'
        }
        emoji = status_emoji.get(detail['status'], '❓')
        
        # Hiển thị thông tin match
        if detail.get('test_case_idx') is not None:
            match_info = ""
            if detail.get('matched_model_idx') is not None:
                match_info = f" → Matched with Model case #{detail['matched_model_idx']}"
            similarity_info = f" (similarity: {detail.get('similarity', 0.0):.4f})" if detail.get('similarity', 0) > 0 else ""
            print(f"   {emoji} GT Test case #{detail['test_case_idx']}: {detail['status']} - {detail['reason']}{match_info}{similarity_info}")
        else:
            # FP case (Model case không có GT tương ứng)
            print(f"   {emoji} Model Test case #{detail.get('matched_model_idx', '?')}: {detail['status']} - {detail['reason']}")

# Ví dụ: xem chi tiết record đầu tiên
show_record_details(0)


In [ ]:
# ================================================================
# 11. PHÂN TÍCH THEO NHÓM (nếu có thông tin phân loại)
# ================================================================

# Phân tích theo domain (nếu có)
if 'domain' in pre_df.columns:
    print("\n" + "=" * 60)
    print("PHÂN TÍCH THEO DOMAIN")
    print("=" * 60)
    domain_stats = pre_df.groupby('domain').agg({
        'precision': 'mean',
        'recall': 'mean',
        'f1': 'mean',
        'tp': 'sum',
        'fp': 'sum',
        'fn': 'sum'
    }).round(4)
    print(domain_stats)

# Phân tích theo api_functionality (nếu có)
if 'api_functionality' in pre_df.columns:
    print("\n" + "=" * 60)
    print("PHÂN TÍCH THEO API FUNCTIONALITY")
    print("=" * 60)
    func_stats = pre_df.groupby('api_functionality').agg({
        'precision': 'mean',
        'recall': 'mean',
        'f1': 'mean',
        'id': 'count'
    }).round(4)
    func_stats.columns = ['avg_precision', 'avg_recall', 'avg_f1', 'count']
    print(func_stats)


In [ ]:
# ================================================================
# 13. VẼ BIỂU ĐỒ PHÂN TÍCH (Optional)
# ================================================================

try:
    import matplotlib.pyplot as plt
    import numpy as np
    
    # Tạo figure với subplots
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('Phân tích Precision, Recall, F1 Score', fontsize=16, fontweight='bold')
    
    # 1. Distribution of Precision, Recall, F1
    ax1 = axes[0, 0]
    ax1.hist(pre_df['precision'].dropna(), bins=20, alpha=0.7, label='Precision', color='blue')
    ax1.hist(pre_df['recall'].dropna(), bins=20, alpha=0.7, label='Recall', color='green')
    ax1.hist(pre_df['f1'].dropna(), bins=20, alpha=0.7, label='F1', color='red')
    ax1.set_xlabel('Score')
    ax1.set_ylabel('Frequency')
    ax1.set_title('Distribution of Metrics')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Scatter plot: Precision vs Recall
    ax2 = axes[0, 1]
    ax2.scatter(pre_df['recall'], pre_df['precision'], alpha=0.6, s=50)
    ax2.set_xlabel('Recall')
    ax2.set_ylabel('Precision')
    ax2.set_title('Precision vs Recall')
    ax2.grid(True, alpha=0.3)
    ax2.set_xlim([-0.05, 1.05])
    ax2.set_ylim([-0.05, 1.05])
    
    # 3. Bar chart: TP, FP, FN totals
    ax3 = axes[1, 0]
    categories = ['TP', 'FP', 'FN']
    values = [total_tp, total_fp, total_fn]
    colors = ['green', 'red', 'orange']
    ax3.bar(categories, values, color=colors, alpha=0.7)
    ax3.set_ylabel('Count')
    ax3.set_title('Total TP, FP, FN')
    ax3.grid(True, alpha=0.3, axis='y')
    for i, v in enumerate(values):
        ax3.text(i, v, str(v), ha='center', va='bottom', fontweight='bold')
    
    # 4. Metrics comparison
    ax4 = axes[1, 1]
    metrics_names = ['Overall\nPrecision', 'Overall\nRecall', 'Overall\nF1', 
                     'Average\nPrecision', 'Average\nRecall', 'Average\nF1']
    metrics_values = [overall_precision, overall_recall, overall_f1,
                      avg_precision, avg_recall, avg_f1]
    colors_metrics = ['blue', 'green', 'red', 'lightblue', 'lightgreen', 'pink']
    bars = ax4.bar(metrics_names, metrics_values, color=colors_metrics, alpha=0.7)
    ax4.set_ylabel('Score')
    ax4.set_title('Overall vs Average Metrics')
    ax4.set_ylim([0, 1.1])
    ax4.grid(True, alpha=0.3, axis='y')
    for bar, val in zip(bars, metrics_values):
        height = bar.get_height()
        ax4.text(bar.get_x() + bar.get_width()/2., height,
                f'{val:.3f}', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.savefig('./dataset_eval/metrics_analysis.png', dpi=150, bbox_inches='tight')
    print("✅ Đã lưu biểu đồ phân tích vào: ./dataset_eval/metrics_analysis.png")
    plt.show()
    
except ImportError:
    print("⚠️ Matplotlib chưa được cài đặt. Bỏ qua phần vẽ biểu đồ.")
    print("   Có thể cài đặt bằng: pip install matplotlib")
except Exception as e:
    print(f"⚠️ Lỗi khi vẽ biểu đồ: {e}")


In [ ]:
# ================================================================
# 12. LƯU KẾT QUẢ CHI TIẾT
# ================================================================

# Tạo dataframe chi tiết từng test case
testcase_details = []
for record_result in detailed_results:
    record_id = record_result['record_id']
    for detail in record_result['details']:
        testcase_details.append({
            'record_id': record_id,
            'test_case_idx': detail['test_case_idx'],
            'status': detail['status'],
            'reason': detail['reason']
        })

testcase_df = pd.DataFrame(testcase_details)

# Lưu kết quả
output_file = "./dataset_eval/evaluation_results.csv"
pre_df[['id', 'precision', 'recall', 'f1', 'tp', 'fp', 'fn']].to_csv(output_file, index=False)
print(f"\n✅ Đã lưu kết quả tổng quan vào: {output_file}")

# Lưu chi tiết từng test case
testcase_output_file = "./dataset_eval/testcase_details.csv"
testcase_df.to_csv(testcase_output_file, index=False)
print(f"✅ Đã lưu chi tiết từng test case vào: {testcase_output_file}")

print(f"\n📊 Tổng số test cases được đánh giá: {len(testcase_df)}")
print(f"   - TP: {len(testcase_df[testcase_df['status'] == 'TP'])}")
print(f"   - FP: {len(testcase_df[testcase_df['status'] == 'FP'])}")
print(f"   - FN: {len(testcase_df[testcase_df['status'] == 'FN'])}")
print(f"   - FP+FN: {len(testcase_df[testcase_df['status'] == 'FP+FN'])}")
